# 대규모 언어 모델

In [2]:
from dotenv import load_dotenv

load_dotenv()


True

## 오픈AI API

In [11]:
import openai
from typing import List

In [12]:
#기본 오픈AI 클라이언트 사용
client = openai.OpenAI()

In [13]:
# "안녕하세요!" 메시지를 보내고 응답을 받음
response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages=[{"role": "user", "content": "안녕하세요!"}],
)
response.choices[0].message.content  # 응답 메시지 내용

'안녕하세요! 어떻게 도와드릴까요?'

In [14]:
# 요청에 사용할 프롬프트 템플릿 정의
prompt_template = "주제 {topic}에 대해 짧은 설명을 해주세요."

In [15]:
# 메시지를 보내고 모델의 응답을 받는 함수
def call_chat_model(messages: List[dict]):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
    )
    return response.choices[0].message.content

In [16]:
# 주어진 주제에 따라 설명을 요청하는 함수
def invoke_chain(topic: str):
    prompt_value = prompt_template.format(topic=topic)
    messages = [{"role": "user", "content": prompt_value}]
    return call_chat_model(messages)
# "더블딥" 주제로 설명 요청
invoke_chain("더블딥")

'더블딥(Double Dip)은 경제학에서 주로 사용되는 용어로, 경기침체 후 회복이 일어났다가 다시 경기침체에 빠지는 현상을 의미합니다. 즉, 일시적인 경기 회복이 있은 후 다시 경제 지표가 악화되어 두 번째 침체가 발생하는 경우를 가리킵니다. 이는 소비자 신뢰나 기업 투자 등이 완전히 회복되지 않은 상태에서 일어날 수 있으며, 일반적으로 경제 정책의 효과를 평가하는 데 중요한 신호로 여겨집니다.'

## 랭체인

In [18]:
# 라이브러리 불러오기
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


In [19]:
# 주어진 주제에 대해 짧은 설명을 요청하는 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_template(
    "주제 {topic}에 대해 짧은 설명을 해주세요."
)

In [20]:
#출력 파서를 문자열로 설정
output_parser = StrOutputParser()
# 오픈AI git-4o-mini 모델을 사용하여 채팅 모델 설정
model = ChatOpenAI(model="gpt-4o-mini")


StrOutputParser() : 모델이 반환하는 응답을 문자열로 처리할 수 있도록 처리, 문자열 응답만 추출하여 반환환

In [21]:
# 파이프라인 설정: 주제를 받아 프롬프트를 생성하고, 모델로 응답을 생성한 후 문자열로 파싱
chain = (
    {"topic": RunnablePassthrough()}    # 입력받은 주제를 그대로 통과시킴
    | prompt    #프롬프트 템플릿 적용
    | model    #모델에 프롬프트를 전달하여 응답 생성
    | output_parser    #응답을 문자열로 파싱
)
# 더블딥 주제로 설명 요청
chain.invoke({"topic": "더블딥"})

'"더블딥"은 경제학에서 사용되는 용어로, 경기 침체가 두 번 발생하는 상황을 의미합니다. 일반적으로 첫 번째 침체 후 약간의 회복이 있긴 하지만, 그 회복이 오래 가지 않고 다시 경기가 하락하는 상황을 지칭합니다. 이러한 현상은 주로 소비자 신뢰, 고용, 투자 등 여러 경제 지표가 불확실한 경우에 발생하며, 이로 인해 경기가 더욱 악화될 수 있습니다. 더블딥은 특히 금융 위기나 경제적 충격이 발생한 후에 자주 나타날 수 있습니다.'

### 오픈AI와 랭체인 비교

|항목|오픈AI API|랭체인|
|:--:|:--:|:--:|
|구조|단순한  호출 방식| 모듈화된 체인구조|
|유연성|낮음:특정 모델 종속|높음: 다양한 모델 전환 및 기능 확장 가능|
|코드 복잡도|간단함: 코드 길이 짧음|체계적이지만 다소 길어질 수 있음|
|재사용성|낮음: 코드 일부를 수정해야 재사용 가능| 높음: 모듈화된 구성으로 손쉽게 재사용 가능|
|사용사례|간단한 작업|복잡하고 확장 가능한 작업|
|모델 전환 용이성|제한적: 코드 수정 필요|매우용이: 모델 클래스를 바꾸는 것만으로 전환 가능|

## 대규모 언어 모델 파라미터 설정
하이퍼파라미터(속성값) : 모델이 생성하는 텍스트의 스타일, 길이, 정확도등에 영향을 줌. 출력을 최적화.
- 온도(temperature) 
  - 생성하는 텍스트의 다양성. 
  - 0~1 사이의 값. 
  - 작을수록 예측 가능하고 클수록 다양하고 창의적인 출력 생성
- 최대 토큰수(Max Tokens)
  - 생성할 최대 토큰수 지정하여 생성할 텍스트의 길이를 제한
- 최상위 P(Top P)
  - 생성 과정에서 특정 확률 분포 내에서 상위 P%의 토큰만을 고려하는 방식
  - 출력의 다양성을 조정하는 데 도움
- 빈도 패널티(Frequency Penalty)
  - 0~1 사이의 값
  - 값이 클수록 이미 등장한 단어나 구절이 다시 등장할 확률을 감소
  - 반복을 줄이고 텍스트의 다양성을 증가
- 존재 패널티(Presence Penalty)
  - 텍스트 내에서 단어의 존재 유무에 따라 해당 단어의 선택 확률을 조정
  - 0~1값
  - 값이 클수록 텍스트에 등장하지 않은 새로운 단어 사용을 장려
- 정지 시퀀스(Stop Sequences)
  - 특정 단어나 구절이 등장할 경우 생성을 멈추도록 설정
  - 출력을 특정 포인트에서 종료하고자 할때 사용

**사용예제**
```python
from langchain_openai import OpenAI
llm = OpenAI(
    temperature=0.7,    #온도 설정(0~1)
    max_tokens=100, #최대 토큰수
    model_name="text-davinci-002"   #사용할 모델
)
```

### LogicKor 리더보드
https://lk.instruct.kr/